In [4]:
import logging

logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s  - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')

In [5]:
from optimex.converter import Converter
from optimex import optimizer
from optimex import postprocessing 
import pandas as pd

con_vs = "v8"
fixed = None
for optimex_type in ["dynamic_t50", "time_explicit_t130"]:
# for optimex_type in ["time_explicit_t50"]:

    converter = Converter(None)
    converter.unpickle_model_inputs(f"data_ma/{optimex_type}_opt_inputs.pkl")

    converter.model_inputs.process_limits_max_default = 0.5e9
    if converter.model_inputs.process_limits_max is None:
        converter.model_inputs.process_limits_max = {}
    for year in range(2020, 2051):
        converter.model_inputs.process_limits_max.update({
            ("heat_prod_bio_gas_cogen", year): 0.5*3.6e9,
            ("heat_prod_wood_chips_cogen", year): 0.5*3.6e9,
            ("heat_prod_natural_gas_boiler", year): 0.5*3.6e9,
            ("heat_prod_fuel_cell_cogen", year): 0.5*3.6e9,
        })
    for year in range(2020, 2041):
        converter.model_inputs.process_limits_max.update({
            ("heat_prod_fuel_cell_cogen", year): 0.0,
        }) 
    if converter.model_inputs.cumulative_process_limits_max is None:
        converter.model_inputs.cumulative_process_limits_max = {}
    converter.model_inputs.cumulative_process_limits_max.update({
        "heat_prod_natural_gas_boiler": 2*3.6e9,
        "heat_prod_wood_chips_cogen": 2*3.6e9,
        "elec_prod_hydro_run_of_river": 0,
        "elec_prod_wind_offshore": 0,
        "elec_prod_hydro_reservoir": 1e9,
        "elec_prod_wind_onshore": 1e9,    
    })

    converter.pickle_model_inputs(f"data_ma/{optimex_type}_opt_with_{con_vs}_con_inputs.pkl")

    if fixed is not None:
        fixed_scaling = pd.read_csv(f"results/{fixed}_{con_vs}_opt_scaling.csv", index_col=0)
        model = optimizer.create_model(converter.model_inputs, name = f"optimex_fixed_{optimex_type}")
        for time, row in fixed_scaling.iterrows():
            for process in model.PROCESS:
                if process in row:  # Ensure the process exists in the DataFrame columns
                    value = row[process]
                    if time in model.SYSTEM_TIME and process in model.PROCESS:
                        model.scaling[process, time].fix(value)
    else:
        model = optimizer.create_model(converter.model_inputs, name = f"optimex_{optimex_type}")
    model, results = optimizer.solve_model(model, compute_iis=True)
    if fixed is not None:
        results.write(filename=f"results/{optimex_type}_{con_vs}_fixed_scaling_{fixed}_results.csv")
    else:
        results.write(filename=f"results/{optimex_type}_{con_vs}_opt_static_results.csv")
    pp = postprocessing.PostProcessor(model)
    df_scaling = pp.get_scaling()
    df_scaling.to_csv(f"results/{optimex_type}_{con_vs}_opt_scaling.csv")
    dfs_production = pp.get_production()
    for key, df in dfs_production.items():
        df.to_csv(f"results/{optimex_type}_{con_vs}_opt_{key}.csv")
    df_demand = pp.get_demand()
    df_demand.to_csv(f"results/{optimex_type}_{con_vs}_opt_demand.csv")


2025-01-25 21:42:07  - INFO - Creating sets
2025-01-25 21:42:07  - INFO - Creating parameters
2025-01-25 21:42:07  - INFO - Creating variables
2025-01-25 21:42:07  - INFO - Creating expressions
2025-01-25 21:47:05  - INFO - Creating constraints
2025-01-25 21:47:05  - INFO - Creating objective function


Set parameter Username
Academic license - for non-commercial use only - expires 2026-01-20
Read LP format model from file C:\Users\LAYTON~1\AppData\Local\Temp\tmpjb795t1y.pyomo.lp
Reading time = 0.02 seconds
x1: 1569 rows, 651 columns, 14108 nonzeros
Set parameter MIPGap to value 0.01
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 1569 rows, 651 columns and 14108 nonzeros
Model fingerprint: 0xd331e91e
Coefficient statistics:
  Matrix range     [2e-02, 1e+01]
  Objective range  [3e-06, 7e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e+07, 7e+09]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 1515 rows and 447 columns
Presolve time: 0.00s
Presolved: 54 rows, 204 columns, 3720 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.112885e+10   0.

2025-01-25 21:48:13  - INFO - Creating sets
2025-01-25 21:48:13  - INFO - Creating parameters
2025-01-25 21:48:13  - INFO - Creating variables
2025-01-25 21:48:13  - INFO - Creating expressions
2025-01-25 22:23:15  - INFO - Creating constraints
2025-01-25 22:23:16  - INFO - Creating objective function


Set parameter Username
Academic license - for non-commercial use only - expires 2026-01-20
Read LP format model from file C:\Users\LAYTON~1\AppData\Local\Temp\tmpqo5rns49.pyomo.lp
Reading time = 0.07 seconds
x1: 4049 rows, 1691 columns, 47696 nonzeros
Set parameter MIPGap to value 0.01
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4049 rows, 1691 columns and 47696 nonzeros
Model fingerprint: 0xc8883633
Coefficient statistics:
  Matrix range     [2e-02, 1e+01]
  Objective range  [9e-07, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e+07, 7e+09]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 3995 rows and 1487 columns
Presolve time: 0.01s
Presolved: 54 rows, 204 columns, 3720 nonzeros

Ordering 

In [6]:
converter.model_inputs.PROCESS

['elec_prod_fuel_cell_cogen',
 'heat_prod_bio_gas_cogen',
 'heat_prod_natural_gas_boiler',
 'elec_prod_bio_gas_cogen',
 'heat_prod_fuel_cell_cogen',
 'elec_prod_lignite',
 'elec_prod_wood_chips_cogen',
 'elec_prod_wind_offshore',
 'elec_prod_wind_onshore',
 'elec_prod_hydro_run_of_river',
 'heat_prod_wood_chips_cogen',
 'elec_prod_photovoltaic',
 'elec_prod_hydro_reservoir']